In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
%matplotlib inline
import imp

import sys
  
# adding utils folder to the system path
sys.path.insert(0, '../utils')
  
# importing the utils_main library as utm
import utils_main as utm

# Read files

In [2]:
portfolio = pd.read_json('./../data/portfolio.json', lines=True)
profile = pd.read_json('./../data/profile.json', lines=True)
transcript = pd.read_json('./../data/transcript.json', lines=True)

The data comprises of three separate file described below.

# Portfolio data

portfolio.json contains information about the ten promotions.
* reward is the monetary value of the promotion.
* channels is the ways in which the promotion was advertised.
* diffculty is the amount the customer needs to spend in order to receive the reward.
* duration is the total number of days that the promotion was available.
* offer_type is the type of promotion. This is either a money off offer (discount), buy one get one free (BOGO) or
a news letter (informational).
* id is the promotion identitifcation --> rename as id_promotion


Preprocessing:
* Created the binary columns from the channels column of type list:
    * email column
    * mobile column
    * social column
    * web column
* Renamed id column into id_promotion column

In [3]:
imp.reload(utm)

<module 'utils_main' from 'C:\\Users\\BASTAMX2\\04_TargetingCustomers_DAIB\\TargetingCustomers_DAIB\\eda\\../utils\\utils_main.py'>

In [4]:
portfolio, profile, transcript = utm.readFiles(dropUnnecessaryCol = True)

In [5]:
portfolio

,reward,difficulty,duration,offer_type,id_promotion,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


In [6]:
portfolio.shape

(10, 9)

In [7]:
print(portfolio.isnull().sum())

reward          0
difficulty      0
duration        0
offer_type      0
id_promotion    0
email           0
mobile          0
social          0
web             0
dtype: int64


In [8]:
print(portfolio.isna().sum())

reward          0
difficulty      0
duration        0
offer_type      0
id_promotion    0
email           0
mobile          0
social          0
web             0
dtype: int64


In [9]:
portfolio.dtypes

reward           int64
difficulty       int64
duration         int64
offer_type      object
id_promotion    object
email            int64
mobile           int64
social           int64
web              int64
dtype: object

In [10]:
portfolio_report = ProfileReport(portfolio)
portfolio_report.to_file("reports/portfolio.html")
#portfolio_report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Profile data

profile.json contains information about customers.
* gender is the identified gender of the customer.
* age is the age of the customer at the time of the promotion period.
* id is the customer membership identification --> rename as id_membership
* became_member is the date when the customer became a member.
* income is the self reported income of the customer at the time of the promotion period.


Assumption:
* You can assume that all customers were members before the promotional period began

Preprocessing:
* Changes format of the "became_member_on" column from int to datetime64[ns]
* Renamed id column into id_membership column

Things to do:
*  Since in this case we want the cluster customers, someone who joined in say March 2015 would have a longer membership than someone who joined say March 2018 and therefore the important aspect here may be the 'length of membership'.

In [11]:
portfolio, profile, transcript = utm.readFiles(dropUnnecessaryCol = True)

In [12]:
profile.head()

,gender,age,id_membership,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


In [13]:
profile.shape

(17000, 5)

In [14]:
profile.dtypes

gender                      object
age                          int64
id_membership               object
became_member_on    datetime64[ns]
income                     float64
dtype: object

In [15]:
print(profile.isnull().sum())

gender              2175
age                    0
id_membership          0
became_member_on       0
income              2175
dtype: int64


In [16]:
print(profile.isna().sum())

gender              2175
age                    0
id_membership          0
became_member_on       0
income              2175
dtype: int64


In [17]:
profile_report = ProfileReport(profile)
profile_report.to_file("reports/profile.html")
#profile_report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Transcript data

transactions.json contains information specific transaction across the promotion period.
* person is the customer membership identification --> renamed id_membership
* event is the event - either offer received, offer viewed, offer complete or transaction.
* value.offer.id is the promotion identification --> renamed id_promotion
* value.amount is the amount spent in GBP for a given transaction.
* value.reward is the monetary value of the promotion.
* time is the time from the beginning of the promotion period.

Preprocessing:
* Extracted values from the "value" dictionnary column into:
    * id_promotion column
    * amount column
    * reward column
* Renamed person column into id_membership column
* Renamed value.offer.id column into id_promotion column

In [18]:
imp.reload(utm)

<module 'utils_main' from 'C:\\Users\\BASTAMX2\\04_TargetingCustomers_DAIB\\TargetingCustomers_DAIB\\eda\\../utils\\utils_main.py'>

In [19]:
portfolio, profile, transcript = utm.readFiles(dropUnnecessaryCol = True)

In [20]:
transcript.head()

,id_membership,event,time,id_promotion,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,0.0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0


In [21]:
transcript["id_promotion"].unique()

array(['9b98b8c7a33c4b65b9aebfe6a799e6d9',
       '0b1e1539f2cc45b7b9fa7c272da2e1d7',
       '2906b810c7d4411798c6938adc9daaa5',
       'fafdcd668e3743c1bb461111dcafc2a4',
       '4d5c57ea9a6940dd891ad53e9dbe8da0',
       'f19421c1d4aa40978ebb69ca19b0e20d',
       '2298d6c36e964ae4a3e7e9706d1fb8c2',
       '3f207df678b143eea3cee63160fa8bed',
       'ae264e3637204a6fb9bb56bc8210ddfd',
       '5a8bc65990b245e5a138643cd4eb9837', ''], dtype=object)

In [22]:
transcript["amount"].unique()

array([  0.  ,   0.83,  34.56, ..., 685.07, 405.04, 476.33])

In [23]:
transcript["reward"].unique()

array([ 0,  2,  5, 10,  3], dtype=int64)

In [24]:
transcript.shape

(306534, 6)

In [25]:
transcript.dtypes

id_membership     object
event             object
time               int64
id_promotion      object
amount           float64
reward             int64
dtype: object

In [26]:
print(transcript.isnull().sum())

id_membership    0
event            0
time             0
id_promotion     0
amount           0
reward           0
dtype: int64


In [27]:
print(transcript.isna().sum())

id_membership    0
event            0
time             0
id_promotion     0
amount           0
reward           0
dtype: int64


In [28]:
transcript_report = ProfileReport(transcript)
transcript_report.to_file("reports/transcript.html")
#transcript_report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\BASTAMX2\Anaconda3\envs\daib\lib\site-packages\scipy\stats\stats.py:4812: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Merging dataframes
This section does some investigation about merging dataframes

The idea is to merge the portfolio, profile, and transcript dataframes using the id_membership and id_promotion keys

In [29]:
portfolio.columns

Index(['reward', 'difficulty', 'duration', 'offer_type', 'id_promotion',
       'email', 'mobile', 'social', 'web'],
      dtype='object')

In [30]:
profile.columns

Index(['gender', 'age', 'id_membership', 'became_member_on', 'income'], dtype='object')

In [31]:
transcript.columns

Index(['id_membership', 'event', 'time', 'id_promotion', 'amount', 'reward'], dtype='object')

In [32]:
trans_pro = pd.merge(transcript, profile, on="id_membership")

In [33]:
transcript.shape

(306534, 6)

In [34]:
trans_pro.shape

(306534, 10)

The transcript df and the merged trans_pro df have the same number of rows --> no data has been lost --> Looks ok

In [35]:
trans_pro.head()

,id_membership,event,time,id_promotion,amount,reward,gender,age,became_member_on,income
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0,F,75,2017-05-09,100000.0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0,F,75,2017-05-09,100000.0
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,132,,19.89,0,F,75,2017-05-09,100000.0
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132,,0.00,5,F,75,2017-05-09,100000.0
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,144,,17.78,0,F,75,2017-05-09,100000.0


In [36]:
trans_pro_port = pd.merge(trans_pro, portfolio, on="id_promotion")

In [37]:
trans_pro.shape

(306534, 10)

In [38]:
trans_pro_port.shape

(134002, 18)

The trans_pro df and the merged trans_pro_port df do NOT have the same number of rows --> some data has been lost --> TO DOUBLE CHECK

In [39]:
trans_pro_port.head()

,id_membership,event,time,id_promotion,amount,reward_x,gender,age,became_member_on,income,reward_y,difficulty,duration,offer_type,email,mobile,social,web
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,F,75,2017-05-09,100000.0,5,5,7,bogo,1,1,0,1
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,F,75,2017-05-09,100000.0,5,5,7,bogo,1,1,0,1
2,e2127556f4f64592b11af22de27a7932,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,M,68,2018-04-26,70000.0,5,5,7,bogo,1,1,0,1
3,e2127556f4f64592b11af22de27a7932,offer viewed,420,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,M,68,2018-04-26,70000.0,5,5,7,bogo,1,1,0,1
4,68617ca6246f4fbc85e91a2a49552598,offer received,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,None,118,2017-10-02,NaN,5,5,7,bogo,1,1,0,1
